<a href="https://colab.research.google.com/github/Ayukhoirunnisa/Belajar/blob/main/Bangkit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!unzip BANGKIT2021

Archive:  BANGKIT2021.zip
   creating: BANGKIT2021/
   creating: BANGKIT2021/trash/
  inflating: BANGKIT2021/trash/trash18.jpg  
  inflating: BANGKIT2021/trash/trash46.jpg  
  inflating: BANGKIT2021/trash/trash10.jpg  
  inflating: BANGKIT2021/trash/trash11.jpg  
  inflating: BANGKIT2021/trash/trash25.jpg  
  inflating: BANGKIT2021/trash/trash70.jpg  
  inflating: BANGKIT2021/trash/trash4.jpg  
  inflating: BANGKIT2021/trash/trash51.jpg  
  inflating: BANGKIT2021/trash/trash23.jpg  
  inflating: BANGKIT2021/trash/trash53.jpg  
  inflating: BANGKIT2021/trash/trash26.jpg  
  inflating: BANGKIT2021/trash/trash91.jpg  
  inflating: BANGKIT2021/trash/trash137.jpg  
  inflating: BANGKIT2021/trash/trash80.jpg  
  inflating: BANGKIT2021/trash/trash87.jpg  
  inflating: BANGKIT2021/trash/trash73.jpg  
  inflating: BANGKIT2021/trash/trash12.jpg  
  inflating: BANGKIT2021/trash/trash47.jpg  
  inflating: BANGKIT2021/trash/trash116.jpg  
  inflating: BANGKIT2021/trash/trash3.jpg  
  inflating: BAN

In [2]:
import os

In [3]:
mypath= 'BANGKIT2021/'

In [4]:
file_name = []
tag = []
full_path = []
for path, subdirs, files in os.walk(mypath):
    for name in files:
        full_path.append(os.path.join(path, name)) 
        tag.append(path.split('/')[-1])        
        file_name.append(name)

In [5]:
import pandas as pd

df = pd.DataFrame({"path":full_path,'file_name':file_name,"tag":tag})
df.groupby(['tag']).size()
df.head()



,path,file_name,tag
0,BANGKIT2021/paper/paper372.jpg,paper372.jpg,paper
1,BANGKIT2021/paper/paper9.jpg,paper9.jpg,paper
2,BANGKIT2021/paper/paper509.jpg,paper509.jpg,paper
3,BANGKIT2021/paper/paper430.jpg,paper430.jpg,paper
4,BANGKIT2021/paper/paper141.jpg,paper141.jpg,paper


In [6]:
from sklearn.model_selection import train_test_split

#variabel yang digunakan pada pemisahan data ini
X= df['path']
y= df['tag']

# split dataset awal menjadi data train dan test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=300)

# kemudian data test dibagi menjadi 2 sehingga menjadi data test dan data validation.
X_test, X_val, y_test, y_val = train_test_split(
    X_test, y_test, test_size=0.5, random_state=100)

# menyatukan kedalam masing-masing dataframe
df_tr = pd.DataFrame({'path':X_train
              ,'tag':y_train
             ,'set':'train'})

df_te = pd.DataFrame({'path':X_test
              ,'tag':y_test
             ,'set':'test'})

df_val = pd.DataFrame({'path':X_val
              ,'tag':y_val
             ,'set':'validation'})

df_all = df_tr.append([df_te,df_val]).reset_index(drop=1)\

In [7]:
import shutil
from tqdm.notebook import tqdm as tq

datasource_path = "BANGKIT2021/"
dataset_path = "dataset/"
for index, row in tq(df_all.iterrows()):
    
    #detect filepath
    file_path = row['path']
    if os.path.exists(file_path) == False:
            file_path = os.path.join(datasource_path,row['tag'],row['image'].split('.')[0])            
    
    #make folder destination dirs
    if os.path.exists(os.path.join(dataset_path,row['set'],row['tag'])) == False:
        os.makedirs(os.path.join(dataset_path,row['set'],row['tag']))
    
    #define file dest
    destination_file_name = file_path.split('/')[-1]
    file_dest = os.path.join(dataset_path,row['set'],row['tag'],destination_file_name)
    
    #copy file from source to dest
    if os.path.exists(file_dest) == False:
        shutil.copy2(file_path,file_dest)

In [8]:
import tensorflow as tf

In [9]:
# Define Input Parameters
dim = (150, 150)
# dim = (15, 15)
channel = (3, )
input_shape = dim + channel

#batch size
batch_size = 16

#Epoch
epoch = 10

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1. / 255,
                                 shear_range=0.2,
                                 zoom_range=0.2,
                                 horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True)

train_generator = train_datagen.flow_from_directory('dataset/train/',
                                                    target_size=dim,
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    shuffle=True)

val_generator = val_datagen.flow_from_directory('dataset/validation/',
                                                target_size=dim,
                                                batch_size=batch_size,
                                                class_mode='categorical',
                                                shuffle=True)

test_generator = test_datagen.flow_from_directory('dataset/test/',
                                                  target_size=dim,
                                                  batch_size=batch_size,
                                                  class_mode='categorical',
                                                  shuffle=True)

num_class = test_generator.num_classes
labels = train_generator.class_indices.keys()

Found 1620 images belonging to 5 classes.
Found 203 images belonging to 5 classes.
Found 203 images belonging to 5 classes.


In [11]:
print(labels)

dict_keys(['cardboard', 'metal', 'paper', 'plastic', 'trash'])


In [12]:
def tf_data_generator(generator, input_shape):
    num_class = generator.num_classes
    tf_generator = tf.data.Dataset.from_generator(
        lambda: generator,
        output_types=(tf.float32, tf.float32),
        output_shapes=([None
                        , input_shape[0]
                        , input_shape[1]
                        , input_shape[2]]
                       ,[None, num_class])
    )
    return tf_generator
train_data = tf_data_generator(train_generator, input_shape)
test_data = tf_data_generator(test_generator, input_shape)
val_data = tf_data_generator(val_generator, input_shape)

In [13]:
from tensorflow.keras.applications import MobileNetV2

# get base models
base_model = MobileNetV2(
    input_shape=input_shape,
    include_top=False,
    weights='imagenet',
    classes=num_class,
)

9412608/9406464 [==============================] - 0s 0us/step


In [14]:
from tensorflow.keras import layers,Sequential
from tensorflow.keras.models import Model

#Adding custom layers
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(1024, activation="relu")(x)

predictions = layers.Dense(num_class, activation="softmax")(x)
model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [15]:
model_viz = tf.keras.utils.plot_model(model,
                          to_file='model.png',
                          show_shapes=True,
                          show_layer_names=True,
                          rankdir='TB',
                          expand_nested=True,
                          dpi=55)

In [16]:
EPOCH = 5
history = model.fit(x=train_data,
        steps_per_epoch=len(train_generator),
        epochs=EPOCH,
        validation_data=val_data,
        validation_steps=len(val_generator),
        shuffle=True,
        verbose = 1)

Epoch 1/5
102/102 [==============================] - 162s 2s/step - loss: 0.9586 - accuracy: 0.6790 - val_loss: 4.7208 - val_accuracy: 0.3990
Epoch 2/5
102/102 [==============================] - 156s 2s/step - loss: 0.7077 - accuracy: 0.7667 - val_loss: 5.6354 - val_accuracy: 0.3350
Epoch 3/5
102/102 [==============================] - 156s 2s/step - loss: 0.5894 - accuracy: 0.7969 - val_loss: 7.3340 - val_accuracy: 0.1724
Epoch 4/5
102/102 [==============================] - 157s 2s/step - loss: 0.4932 - accuracy: 0.8395 - val_loss: 6.8078 - val_accuracy: 0.1823
Epoch 5/5
102/102 [==============================] - 155s 2s/step - loss: 0.4524 - accuracy: 0.8451 - val_loss: 5.8749 - val_accuracy: 0.3399


In [17]:
import requests
from io import BytesIO
from PIL import Image
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

#Parameters
input_size = (150,150)

#define input shape
channel = (3,)
input_shape = input_size + channel

#define labels
labels = ['cardboard', 'glass', 'metal', 'paper', 'plastic']

def preprocess(img,input_size):
    nimg = img.convert('RGB').resize(input_size, resample= 0)
    img_arr = (np.array(nimg))/255
    return img_arr

def reshape(imgs_arr):
    return np.stack(imgs_arr, axis=0)

for fn in uploaded.keys():
  # predicting images
  path = '/content/' + fn
  im = Image.open(path)
  X = preprocess(im,input_size)
  X = reshape([X])
  y = model.predict(X)
  result = labels[np.argmax(y)]
  print(result)

Saving metal2.jpg to metal2.jpg
metal


In [18]:
search = print('DIY from ' + result)

DIY from metal
